#### This project creates an Apache Cassandra etl pipeline named sparkify_cassandra for the music app, Sparkify. The data is modeled on the following 3 queries:

artist, song_title and song_length for sessionId=338, and itemInSession=4

artist, song (sorted by itemInSession) and user (firstName and lastName) for userid=10, sessionid=182

every user's firstName and lastName for the song 'All Hands Against His Own'

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.cluster import Cluster

#### Creates list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Process the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 

        csvreader = csv.reader(csvfile) 
        next(csvreader)                
        for line in csvreader:
            full_data_rows_list.append(line) 
        
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

3831


#### Creates Cluster

In [5]:
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)



#### Creates Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify_cassandra 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify_cassandra')
except Exception as e:
    print(e)

## Query 1
#### Get the artist, song title and song's length in the music app history that was heard during 

In [8]:
query = "CREATE TABLE IF NOT EXISTS songs_by_session_and_item"
query = query + """(sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, 
                    PRIMARY KEY(sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session_and_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to verify that the data have been inserted into each table

In [10]:
# Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM songs_by_session_and_item WHERE sessionId=338 AND itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
pd.concat([pd.DataFrame([row], columns=['Artist', 'Song', 'Length']) 
                                for row in rows] ,ignore_index=True)


,Artist,Song,Length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query 2
#### Gets name of artist, song sorted by itemInSession and user first and last name

In [11]:

query = "CREATE TABLE IF NOT EXISTS user_by_name_and_session"
query = query + """(userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT,
                    lastName TEXT, PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_by_name_and_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
       


In [13]:
query = """SELECT artist, song, firstName, lastName FROM user_by_name_and_session
            WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.concat([pd.DataFrame([row], columns=['Artist', 'Song', 'First Name', 'Last Name']) 
                                                   for row in rows],ignore_index=True)
                    

,Artist,Song,First Name,Last Name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query 3
#### Gets user name first and last in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
query = "CREATE TABLE IF NOT EXISTS user_by_firstname_and_lastname"
query = query + """(song TEXT, firstName TEXT, lastName TEXT, userId INT,
                    PRIMARY KEY (song, firstName, lastName, userId))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO  user_by_firstname_and_lastname (song, firstName, lastName, userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))
       


In [16]:
query = """SELECT firstName, lastName FROM  user_by_firstname_and_lastname
            WHERE song = 'All Hands Against His Own'"""
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.concat([pd.DataFrame([row], columns=['First Name', 'Last Name']) 
                                for row in rows],ignore_index=True)
                    

,First Name,Last Name
0,Jacqueline,Lynch
1,Sara,Johnson


### Drops the Tables before closing connection¶

In [17]:
try:
    session.execute("DROP TABLE songs_by_session_and_item")
    session.execute("DROP TABLE user_by_name_and_session")
    session.execute("DROP TABLE user_by_firstname_and_lastname")
except Exception as e:
    print(e)

### Closes the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()